<a href="https://colab.research.google.com/github/SamuelJucker/MACHINE_LEARNING_JUCKESAM/blob/master/custom_finnews_summarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Preparation:**

Install the following libraries

In [1]:
!pip install transformers datasets
!pip install transformers[torch] accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.1/542.1 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 22.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.7 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.31.0, but you have requests 2.32.3 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 9.2 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86

Necessary imports // if not acess to drive just comment it out

In [2]:

# Import necessary libraries
import transformers
import accelerate
import torch
import shutil
import os
from transformers import Trainer, TrainingArguments, AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, GenerationConfig
from datasets import load_dataset
from google.colab import drive

import random  # Add this line to import the random module
import pandas as pd  # Ensure pandas is imported for DataFrame usage
from datasets import Dataset




loading / model and tokenizer

In [ ]:

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, GenerationConfig
import shutil
import os

# Step 1: Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Define the Save Path
drive_save_path = '/content/drive/My Drive/fine_tuned_model'

# Load the pretrained model and tokenizer
model_name = "google/pegasus-large"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
base_model = model_name.split('/')[1]  # Extract the base model name for later use

# Define the directory to save the model and tokenizer
save_directory = "./pretrained_pegasus_large"

# Save the pretrained model and tokenizer to the directory
tokenizer.save_pretrained(save_directory)
model.save_pretrained(save_directory)

# Print all files in the save directory
# print("Files in the save directory:")
# for root, dirs, files in os.walk(save_directory):
#     for filename in files:
#         print(os.path.join(root, filename))

# Assuming you have already fine-tuned the model and tokenizer
model.save_pretrained('./results')
tokenizer.save_pretrained('./results')

# Optionally, save the generation configuration if used
gen_config = GenerationConfig(
    max_length=142,
    min_length=56,
    early_stopping=True,
    num_beams=4,
    length_penalty=2.0,
    no_repeat_ngram_size=3,
    forced_bos_token_id=0,
    forced_eos_token_id=2
)
gen_config.save_pretrained('./results')

# List of files to keep (fine-tuned files)
files_to_keep = [
    "config.json",
    "generation_config.json",
    "model.safetensors",  # or 'pytorch_model.bin'
    "special_tokens_map.json",
    "spiece.model",
    "tokenizer.json",
    "tokenizer_config.json"
]

# Print all files in the fine-tuned directory to ensure they are saved
print("Files in the fine-tuned directory:")
for root, dirs, files in os.walk('./results'):
    for filename in files:
        print(os.path.join(root, filename))

# Step 4: Copy Only Fine-Tuned Files to Google Drive
# If the directory exists, delete it first
if os.path.exists(drive_save_path):
    shutil.rmtree(drive_save_path)

# Create the directory in Google Drive
os.makedirs(drive_save_path)

# Copy only the necessary files to Google Drive
for file_name in files_to_keep:
    src_path = os.path.join('./results', file_name)
    dest_path = os.path.join(drive_save_path, file_name)
    if os.path.exists(src_path):
        shutil.copy(src_path, dest_path)
        print(f"Copied {src_path} to {dest_path}")
    else:
        print(f"{src_path} does not exist")

# Print all files in the Google Drive save directory to ensure they are saved
print("Files in the Google Drive save directory:")
for root, dirs, files in os.walk(drive_save_path):
    for filename in files:
        print(os.path.join(root, filename))



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-large and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 256, 'num_beams': 8, 'length_penalty': 0.8, 'forced_eos_token_id': 1}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will 

Files in the fine-tuned directory:
./results/config.json
./results/model.safetensors
./results/tokenizer.json
./results/spiece.model
./results/special_tokens_map.json
./results/tokenizer_config.json
./results/generation_config.json
./results/checkpoint-500/config.json
./results/checkpoint-500/model.safetensors
./results/checkpoint-500/trainer_state.json
./results/checkpoint-500/optimizer.pt
./results/checkpoint-500/training_args.bin
./results/checkpoint-500/scheduler.pt
./results/checkpoint-500/generation_config.json
./results/checkpoint-500/rng_state.pth
./results/checkpoint-1000/config.json
./results/checkpoint-1000/model.safetensors
./results/checkpoint-1000/trainer_state.json
./results/checkpoint-1000/optimizer.pt
./results/checkpoint-1000/training_args.bin
./results/checkpoint-1000/scheduler.pt
./results/checkpoint-1000/generation_config.json
./results/checkpoint-1000/rng_state.pth
./results/checkpoint-1500/config.json
./results/checkpoint-1500/model.safetensors
./results/checkpoi

take a look at this: https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model

Dataload / data preproc etc not in collab

# Define the dataset name
dataset_name = "this can vary"


In [4]:
# dataset_name = "cnn_dailymail"
# dataset_version = "3.0.0"


Load temporary dataset, currently temp future final temp down.

In [5]:
import json
import pandas as pd
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import random

# Load the JSON files
train_json_file_path = '/content/drive/My Drive/Data_ML2/ECTSum_train.jsonl'
val_json_file_path = '/content/drive/My Drive/Data_ML2/ECTSum_val.jsonl'
test_json_file_path = '/content/drive/My Drive/Data_ML2/ECTSum_test.jsonl'

# Read the JSONL files
train_data = []
with open(train_json_file_path, 'r') as file:
    for line in file:
        train_data.append(json.loads(line))

val_data = []
with open(val_json_file_path, 'r') as file:
    for line in file:
        val_data.append(json.loads(line))

test_data = []
with open(test_json_file_path, 'r') as file:
    for line in file:
        test_data.append(json.loads(line))
train_df = pd.DataFrame(train_data)  # Added this line to define train_df
val_df = pd.DataFrame(val_data)      # Added this line to define val_df
test_df = pd.DataFrame(test_data)    # Added this line to define test_df

# Convert to DataFrames for better handling
train_dataset = Dataset.from_pandas(train_df)
validation_dataset = Dataset.from_pandas(val_df)
test_dataset = Dataset.from_pandas(test_df)


# Combine the datasets for preprocessing and fine-tuning
combined_df = pd.concat([train_df, val_df, test_df], ignore_index=True)

# Retrieve a random sample
random_index = random.randint(0, len(combined_df) - 1)
initial_sample = combined_df.iloc[random_index]
print("Random sample from the initial dataset (before processing):")
print(initial_sample)

# Convert DataFrame to Dataset
dataset = Dataset.from_pandas(combined_df)

# Load the tokenizer
model_name = "google/pegasus-large"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Define a preprocessing function
def preprocess_function(examples):
    inputs = [doc for doc in examples["input"]]
    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding="max_length")
    labels = tokenizer(text_target=examples["summary"], max_length=128, truncation=True, padding="max_length")
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Apply the preprocessing function to the dataset
processed_dataset = dataset.map(preprocess_function, batched=True)

# Retrieve the processed sample
processed_sample = processed_dataset[random_index]
print("\nRandom sample from the processed dataset (after processing):")
print(processed_sample)


Random sample from the initial dataset (before processing):
ticker                                                    FL
quarter                                                   q1
year                                                    2021
input      On a per share basis, first quarter earnings w...
summary    foot locker q1 non-gaap earnings per share $1....
Name: 2100, dtype: object


Map:   0%|          | 0/2425 [00:00<?, ? examples/s]


Random sample from the processed dataset (after processing):
{'ticker': 'FL', 'quarter': 'q1', 'year': '2021', 'input': "On a per share basis, first quarter earnings were $1.93 compared to a loss per share of $1.06 last year and earnings per share of $1.52 for the first quarter of 2019.\nThis year's quarter includes pre-tax charges of $2 million related to the impairment of one of the company's minority investments, and $2 million primarily related to severance costs in connection with the reorganization of certain support functions.\nExcluding these items, first quarter non-GAAP earnings were $1.96 per share, a significant reversal to the loss per share of $0.67 for the first quarter of last year and up 28.1% compared to earnings per share of $1.53 for the first quarter of 2019.\nWe're excited to have him on board.\nLast quarter, I talked about entering our fiscal 2021 momentum.\nThat was certainly the case in our first quarter.\nAnd the momentum gained strength as the quarter progre

In [28]:
import json
import pandas as pd
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import random
import os
import shutil

# Load the JSON files
train_json_file_path = '/content/drive/My Drive/Data_ML2/ECTSum_train.jsonl'
val_json_file_path = '/content/drive/My Drive/Data_ML2/ECTSum_val.jsonl'
test_json_file_path = '/content/drive/My Drive/Data_ML2/ECTSum_test.jsonl'

# Read the JSONL files
train_data = []
with open(train_json_file_path, 'r') as file:
    for line in file:
        train_data.append(json.loads(line))

val_data = []
with open(val_json_file_path, 'r') as file:
    for line in file:
        val_data.append(json.loads(line))

test_data = []
with open(test_json_file_path, 'r') as file:
    for line in file:
        test_data.append(json.loads(line))

# Convert to DataFrames for better handling
train_df = pd.DataFrame(train_data)
val_df = pd.DataFrame(val_data)
test_df = pd.DataFrame(test_data)

# Convert DataFrames to Datasets
train_dataset = Dataset.from_pandas(train_df)
validation_dataset = Dataset.from_pandas(val_df)
test_dataset = Dataset.from_pandas(test_df)

# Combine the datasets for preprocessing and fine-tuning
combined_df = pd.concat([train_df, val_df, test_df], ignore_index=True)

# Retrieve a random sample
random_index = random.randint(0, len(combined_df) - 1)
initial_sample = combined_df.iloc[random_index]
print("Random sample from the initial dataset (before processing):")
print(initial_sample)

# Convert DataFrame to Dataset
dataset = Dataset.from_pandas(combined_df)

# Load the tokenizer and model
model_name = "google/pegasus-large"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# Define a preprocessing function
def preprocess_function(examples):
    inputs = [doc for doc in examples["input"]]
    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding="max_length")
    labels = tokenizer(text_target=examples["summary"], max_length=128, truncation=True, padding="max_length")
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Apply the preprocessing function to the dataset
processed_dataset = dataset.map(preprocess_function, batched=True)

# Retrieve the processed sample
processed_sample = processed_dataset[random_index]
print("\nRandom sample from the processed dataset (after processing):")
print(processed_sample)

# Copy the zipped model to Google Drive with appropriate naming
def copy_to_drive_with_naming(base_name, drive_path):
    file_index = 0
    while True:
        file_name = f"{base_name}{file_index}.zip"
        destination_path = os.path.join(drive_path, file_name)
        if not os.path.exists(destination_path):
            shutil.copyfile('model.zip', destination_path)
            print(f"Model saved as: {file_name}")
            break
        file_index += 1

# Extract the base model name and dataset short name to use as part of the base name
base_model = model_name.split('/')[-1]  # Extract 'pegasus-large' from 'google/pegasus-large'
dataset_short_name = "ECTSum"  # Shortened name for your dataset
base_name = f"model_{base_model}_{dataset_short_name}_"
copy_to_drive_path = "/content/drive/My Drive/Models_ML2"
copy_to_drive_with_naming(base_name, copy_to_drive_path)


Random sample from the initial dataset (before processing):
ticker                                                   EAT
quarter                                                   q1
year                                                    2021
input      As usual, Wyman and Joe will first make prepar...
summary    brinker international inc - qtrly net income p...
Name: 456, dtype: object


Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-large and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/2425 [00:00<?, ? examples/s]


Random sample from the processed dataset (after processing):
{'ticker': 'EAT', 'quarter': 'q1', 'year': '2021', 'input': "As usual, Wyman and Joe will first make prepared comments related to our operating performance and strategic initiatives.\nDuring our call, management may discuss certain items, which are not based entirely on historical facts.\nAnd of course, on the call, we may refer to certain non-GAAP financial measures that management uses in its review of the business, and believes will provide insight into the company's ongoing operations.\nLooking broadly at the quarter, we're encouraged by the continued improvement in the environment, the consumers increasing engagement with the category, and we hope to see those trends continue.\nWe know there are still challenges out there, especially with independents, yet Brinker continues its strong recovery, posting a better-than-expected first quarter and also delivering earnings of $0.28 a share.\nBoth brands increased their progre

Preprocess Data

In [6]:

print(train_dataset.column_names)

train_dataset = train_dataset.map(preprocess_function, batched=True)
validation_dataset = validation_dataset.map(preprocess_function, batched=True)
test_dataset = test_dataset.map(preprocess_function, batched=True)


['ticker', 'quarter', 'year', 'input', 'summary']


Map:   0%|          | 0/1681 [00:00<?, ? examples/s]

Map:   0%|          | 0/249 [00:00<?, ? examples/s]

Map:   0%|          | 0/495 [00:00<?, ? examples/s]

sample after

In [7]:

# Inspect a random sample from the preprocessed dataset
random_index = random.randint(0, len(train_dataset) - 1)
preprocessed_sample = train_dataset[random_index]
print("Random sample from the preprocessed dataset:")
print(pd.DataFrame([preprocessed_sample]))

Random sample from the preprocessed dataset:
  ticker quarter  year                                              input  \
0    JBL      q3  2021  To follow along with slides, please visit jabi...   

                                             summary  \
0  compname posts q3 gaap earnings per share $1.1...   

                                           input_ids  \
0  [413, 857, 466, 122, 9538, 108, 528, 558, 3995...   

                                      attention_mask  \
0  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...   

                                              labels  
0  [12009, 7982, 2131, 15593, 726, 17031, 10384, ...  


In [8]:
# # Preprocess the data
# def preprocess_function(examples):
#     inputs = [doc for doc in examples["article"]]
#     model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding="max_length")  # Reduced max_length
#     labels = tokenizer(text_target=examples["highlights"], max_length=128, truncation=True, padding="max_length")
#     model_inputs["labels"] = labels["input_ids"]
#     return model_inputs

# train_dataset = train_dataset.map(preprocess_function, batched=True)
# validation_dataset = validation_dataset.map(preprocess_function, batched=True)
# test_dataset = test_dataset.map(preprocess_function, batched=True)


In [9]:
# Use DataCollatorForSeq2Seq to handle padding
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)


## **training**

Training / modeling part.... //

In [10]:
# Define training arguments with mixed precision and gradient accumulation
training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=2e-5,
    per_device_train_batch_size=8,  # Adjust batch size if necessary
    per_device_eval_batch_size=8,
    num_train_epochs=12,  # Increase number of epochs for better learning
    weight_decay=0.01,
    fp16=True,  # Enable mixed precision training
    gradient_accumulation_steps=2,  # Accumulate gradients over 2 steps
    logging_steps=100,  # Reduce logging frequency
    save_steps=500,  # Adjust save steps
    evaluation_strategy="steps",  # Evaluate every save step
    eval_steps=500,  # Adjust evaluation steps
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [11]:
# #done for project: (just better more computing power)
# # Define training arguments with mixed precision and gradient accumulation
# training_args = TrainingArguments(
#     output_dir="./results",
#     learning_rate=2e-5,
#     per_device_train_batch_size=8,  # Adjust batch size if necessary
#     per_device_eval_batch_size=8,
#     num_train_epochs=3,  # Increase number of epochs for better learning
#     weight_decay=0.01,
#     fp16=True,  # Enable mixed precision training
#     gradient_accumulation_steps=2,  # Accumulate gradients over 2 steps
#     logging_steps=100,  # Reduce logging frequency
#     save_steps=500,  # Adjust save steps
#     evaluation_strategy="steps",  # Evaluate every save step
#     eval_steps=500,  # Adjust evaluation steps
#     load_best_model_at_end=True,
#     metric_for_best_model="eval_loss",
# )


In [12]:
# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=validation_dataset,
    data_collator=data_collator,
)


In [13]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)


checks:

to do parameter finetuning and: Instruction Tuning is Key: For zero-shot summarization tasks, instruction tuning significantly outperforms mere model scaling.

In [14]:
# Ensure model is on the correct device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


PegasusForConditionalGeneration(
  (model): PegasusModel(
    (shared): Embedding(96103, 1024, padding_idx=0)
    (encoder): PegasusEncoder(
      (embed_tokens): Embedding(96103, 1024, padding_idx=0)
      (embed_positions): PegasusSinusoidalPositionalEmbedding(1024, 1024)
      (layers): ModuleList(
        (0-15): 16 x PegasusEncoderLayer(
          (self_attn): PegasusAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): ReLU()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
          (final_layer_no

Train

wtf comp to other

In [15]:
# Train the model
# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=processed_dataset,
    eval_dataset=processed_dataset,
    data_collator=data_collator,
)
trainer.train()


Step,Training Loss,Validation Loss
500,2.756400,1.102212
1000,0.881000,0.765896
1500,0.823100,0.735025


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 256, 'num_beams': 8, 'length_penalty': 0.8, 'forced_eos_token_id': 1}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 256, 'num_beams': 8, 'length_penalty': 0.8, 'forced_eos_token_id': 1}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-

TrainOutput(global_step=1824, training_loss=2.121599143011528, metrics={'train_runtime': 913.1352, 'train_samples_per_second': 31.868, 'train_steps_per_second': 1.998, 'total_flos': 4.20417072267264e+16, 'train_loss': 2.121599143011528, 'epoch': 12.0})

Vorlage:

In [16]:
import shutil
import os

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Define the Save Path
drive_save_path = '/content/drive/My Drive/fine_tuned_model'

# Load the pretrained model and tokenizer
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, GenerationConfig

model_name = "google/pegasus-large"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# Assume `model` is your fine-tuned model and `tokenizer` is your tokenizer
# Define the local directory to save the fine-tuned model and tokenizer
fine_tuned_directory = "./results"

# Save the fine-tuned model and tokenizer to the local directory
model.save_pretrained(fine_tuned_directory)
tokenizer.save_pretrained(fine_tuned_directory)

# Optionally, save the generation configuration if used
gen_config = GenerationConfig(
    max_length=142,
    min_length=56,
    early_stopping=True,
    num_beams=4,
    length_penalty=2.0,
    no_repeat_ngram_size=3,
    forced_bos_token_id=0,
    forced_eos_token_id=2
)
gen_config.save_pretrained(fine_tuned_directory)
# Print all files in the fine-tuned directory to ensure they are saved
print("Files in the fine-tuned directory:")
for root, dirs, files in os.walk(fine_tuned_directory):
    for filename in files:
        print(os.path.join(root, filename))

# List of files to keep (fine-tuned files)
files_to_keep = [
    "config.json",
    "generation_config.json",
    "model.safetensors",  # or 'pytorch_model.bin'
    "special_tokens_map.json",
    "spiece.model",
    "tokenizer.json",
    "tokenizer_config.json"
]

# Print all files in the fine-tuned directory to ensure they are saved
print("Files in the fine-tuned directory:")
for root, dirs, files in os.walk(fine_tuned_directory):
    for filename in files:
        print(os.path.join(root, filename))

# Step 4: Copy Only Fine-Tuned Files to Google Drive
# If the directory exists, delete it first
if os.path.exists(drive_save_path):
    shutil.rmtree(drive_save_path)

# Create the directory in Google Drive
os.makedirs(drive_save_path)

# Copy only the necessary files to Google Drive
for file_name in files_to_keep:
    src_path = os.path.join(fine_tuned_directory, file_name)
    dest_path = os.path.join(drive_save_path, file_name)
    if os.path.exists(src_path):
        shutil.copy(src_path, dest_path)
        print(f"Copied {src_path} to {dest_path}")
    else:
        print(f"{src_path} does not exist")

# Print all files in the Google Drive save directory to ensure they are saved
print("Files in the Google Drive save directory:")
for root, dirs, files in os.walk(drive_save_path):
    for filename in files:
        print(os.path.join(root, filename))


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-large and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 256, 'num_beams': 8, 'length_penalty': 0.8, 'forced_eos_token_id': 1}


Files in the fine-tuned directory:
./results/config.json
./results/model.safetensors
./results/tokenizer.json
./results/spiece.model
./results/special_tokens_map.json
./results/tokenizer_config.json
./results/generation_config.json
./results/checkpoint-500/config.json
./results/checkpoint-500/model.safetensors
./results/checkpoint-500/trainer_state.json
./results/checkpoint-500/optimizer.pt
./results/checkpoint-500/training_args.bin
./results/checkpoint-500/scheduler.pt
./results/checkpoint-500/generation_config.json
./results/checkpoint-500/rng_state.pth
./results/checkpoint-1000/config.json
./results/checkpoint-1000/model.safetensors
./results/checkpoint-1000/trainer_state.json
./results/checkpoint-1000/optimizer.pt
./results/checkpoint-1000/training_args.bin
./results/checkpoint-1000/scheduler.pt
./results/checkpoint-1000/generation_config.json
./results/checkpoint-1000/rng_state.pth
./results/checkpoint-1500/config.json
./results/checkpoint-1500/model.safetensors
./results/checkpoi

Imports:




In [17]:
# Check the versions of transformers, accelerate, and torch
print("Transformers version:", transformers.__version__)
print("Accelerate version:", accelerate.__version__)
print("Torch version:", torch.__version__)


Transformers version: 4.41.2
Accelerate version: 0.31.0
Torch version: 2.3.0+cu121


**evaluation**

In [18]:
# Evaluate the model (short overvie numbers)
metrics = trainer.evaluate(eval_dataset=test_dataset)
print(metrics)


{'eval_loss': 0.7254868149757385, 'eval_runtime': 3.7487, 'eval_samples_per_second': 132.047, 'eval_steps_per_second': 16.539, 'epoch': 12.0}


Save Configuaration from finetiuned /tempraus

Libraries for evaluation

EVALUATION CURRENTLY WRONG

In [19]:
!pip install rouge_score evaluate

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 4.3 MB/s eta 0:00:00
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=1b367a226aa68531627df073242a63196ac12a3b07990d683fc58d3af6f2f9ad
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


Try Out:

In [20]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# Load the pretrained model and tokenizer
model_name = "google/pegasus-large"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# Example text
text = """
The fake industrial Revolution was a period of major industrialization and innovation that took place during the late 1700s and early 1800s. It began in Great Britain and quickly spread throughout the world. This period marked a significant turning point in history; almost every aspect of daily life was influenced in some way. Improvements in farming techniques and livestock breeding led to increased food production and the rise of the steam engine. The rise of factories created a new class of wealthy industrialists and businessmen, but also created social tensions and unrest. Despite many challenges and hardships, the Industrial Revolution was a period of significant change and transformation, paving the way for the modern industrial economy.
"""

# Tokenize the input text
inputs = tokenizer(text, return_tensors="pt", max_length=512, truncation=True)

# Generate summary
summary_ids = model.generate(inputs["input_ids"], max_length=142, min_length=56, early_stopping=True, num_beams=4, length_penalty=2.0, no_repeat_ngram_size=3)
summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

print(summary)


Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-large and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


This period marked a significant turning point in history; almost every aspect of daily life was influenced in some way. Despite many challenges and hardships, the Industrial Revolution was a period of significant change and transformation, paving the way for the modern industrial economy. The rise of factories created a new class of wealthy industrialists and businessmen, but also created social tensions and unrest. Improvements in farming techniques and livestock breeding led to increased food production and the rise of the steam engine.


try out with paths

In [34]:
import os
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# Load the pretrained model and tokenizer
model_name = "google/pegasus-large"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# Example text
text = """
The fake industrial Revolution was a period of major industrialization and innovation that took place during the late 1700s and early 1800s. It began in Great Britain and quickly spread throughout the world. This period marked a significant turning point in history; almost every aspect of daily life was influenced in some way. Improvements in farming techniques and livestock breeding led to increased food production and the rise of the steam engine. The rise of factories created a new class of wealthy industrialists and businessmen, but also created social tensions and unrest. Despite many challenges and hardships, the Industrial Revolution was a period of significant change and transformation, paving the way for the modern industrial economy.
"""

# Tokenize the input text
inputs = tokenizer(text, return_tensors="pt", max_length=512, truncation=True)

# Generate summary
summary_ids = model.generate(inputs["input_ids"], max_length=142, min_length=56, early_stopping=True, num_beams=4, length_penalty=2.0, no_repeat_ngram_size=3)
summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

print(summary)

# # Print all files in the current directory
# print("Files in the current directory:")
# for root, dirs, files in os.walk("."):
#     for filename in files:
#         print(os.path.join(root, filename))


Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-large and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


This period marked a significant turning point in history; almost every aspect of daily life was influenced in some way. Despite many challenges and hardships, the Industrial Revolution was a period of significant change and transformation, paving the way for the modern industrial economy. The rise of factories created a new class of wealthy industrialists and businessmen, but also created social tensions and unrest. Improvements in farming techniques and livestock breeding led to increased food production and the rise of the steam engine.


In [22]:
import os

# Print all files in the results directory
print("Files in the results directory:")
for root, dirs, files in os.walk("./results"):
    for filename in files:
        print(os.path.join(root, filename))


Files in the results directory:
./results/config.json
./results/model.safetensors
./results/tokenizer.json
./results/spiece.model
./results/special_tokens_map.json
./results/tokenizer_config.json
./results/generation_config.json
./results/checkpoint-500/config.json
./results/checkpoint-500/model.safetensors
./results/checkpoint-500/trainer_state.json
./results/checkpoint-500/optimizer.pt
./results/checkpoint-500/training_args.bin
./results/checkpoint-500/scheduler.pt
./results/checkpoint-500/generation_config.json
./results/checkpoint-500/rng_state.pth
./results/checkpoint-1000/config.json
./results/checkpoint-1000/model.safetensors
./results/checkpoint-1000/trainer_state.json
./results/checkpoint-1000/optimizer.pt
./results/checkpoint-1000/training_args.bin
./results/checkpoint-1000/scheduler.pt
./results/checkpoint-1000/generation_config.json
./results/checkpoint-1000/rng_state.pth
./results/checkpoint-1500/config.json
./results/checkpoint-1500/model.safetensors
./results/checkpoint-

In [23]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, GenerationConfig

# Load the fine-tuned model and tokenizer
model_directory = "./results"  # Path to your fine-tuned model directory
tokenizer = AutoTokenizer.from_pretrained(model_directory)
model = AutoModelForSeq2SeqLM.from_pretrained(model_directory)
gen_config = GenerationConfig.from_pretrained(model_directory)

# Set decoder_start_token_id if not already set
if gen_config.decoder_start_token_id is None:
    gen_config.decoder_start_token_id = model.config.decoder_start_token_id
if gen_config.bos_token_id is None:
    gen_config.bos_token_id = model.config.bos_token_id

# Example text
text = """
The Industrial Revolution was a period of major industrialization and innovation that took place during the late 1700s and early 1800s. It began in Great Britain and quickly spread throughout the world. This period marked a significant turning point in history; almost every aspect of daily life was influenced in some way. Improvements in farming techniques and livestock breeding led to increased food production and the rise of the steam engine. The rise of factories created a new class of wealthy industrialists and businessmen, but also created social tensions and unrest. Despite many challenges and hardships, the Industrial Revolution was a period of significant change and transformation, paving the way for the modern industrial economy.
"""

# Tokenize the input text
inputs = tokenizer(text, return_tensors="pt", max_length=512, truncation=True)

# Generate summary using fine-tuned model
summary_ids = model.generate(
    inputs["input_ids"],
    max_length=gen_config.max_length,
    min_length=gen_config.min_length,
    early_stopping=gen_config.early_stopping,
    num_beams=gen_config.num_beams,
    length_penalty=gen_config.length_penalty,
    no_repeat_ngram_size=gen_config.no_repeat_ngram_size,
    forced_bos_token_id=gen_config.forced_bos_token_id,
    forced_eos_token_id=gen_config.forced_eos_token_id,
    decoder_start_token_id=gen_config.decoder_start_token_id,
    bos_token_id=gen_config.bos_token_id
)
summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

print(summary)


This period marked a significant turning point in history; almost every aspect of daily life was influenced in some way. Despite many challenges and hardships, the Industrial Revolution was a period of significant change and transformation, paving the way for the modern industrial economy.The Industrial Revolution began in the late 1700s and early 1800s. The Industrial Revolution is the period of major industrialization and innovation that began in Great Britain and quickly spread throughout the world.Despite many challenges, hardships and changes in the industrial economy, the industrial revolution was a significant period of change, transformation and innovation.The industrial revolution began in Britain in the early 1700s, and has since spread throughout Europe and the


do some kind of check / just numbers not whole valuation

Finishing Project Steps / Model Storage etc: no need to execute

In [24]:
# Save the model and tokenizer
model.save_pretrained('./results')
tokenizer.save_pretrained('./results')
gen_config = GenerationConfig(
    max_length=142,
    min_length=56,
    early_stopping=True,
    num_beams=4,
    length_penalty=2.0,
    no_repeat_ngram_size=3,
    forced_bos_token_id=0,
    forced_eos_token_id=2
)
gen_config.save_pretrained(fine_tuned_directory)

# Print all files in the fine-tuned directory to ensure they are saved
print("Files in the fine-tuned directory:")
for root, dirs, files in os.walk(fine_tuned_directory):
    for filename in files:
        print(os.path.join(root, filename))

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 256, 'num_beams': 8, 'length_penalty': 0.8, 'forced_eos_token_id': 1}


Files in the fine-tuned directory:
./results/config.json
./results/model.safetensors
./results/tokenizer.json
./results/spiece.model
./results/special_tokens_map.json
./results/tokenizer_config.json
./results/generation_config.json
./results/checkpoint-500/config.json
./results/checkpoint-500/model.safetensors
./results/checkpoint-500/trainer_state.json
./results/checkpoint-500/optimizer.pt
./results/checkpoint-500/training_args.bin
./results/checkpoint-500/scheduler.pt
./results/checkpoint-500/generation_config.json
./results/checkpoint-500/rng_state.pth
./results/checkpoint-1000/config.json
./results/checkpoint-1000/model.safetensors
./results/checkpoint-1000/trainer_state.json
./results/checkpoint-1000/optimizer.pt
./results/checkpoint-1000/training_args.bin
./results/checkpoint-1000/scheduler.pt
./results/checkpoint-1000/generation_config.json
./results/checkpoint-1000/rng_state.pth
./results/checkpoint-1500/config.json
./results/checkpoint-1500/model.safetensors
./results/checkpoi

In [25]:
# Zip the results directory, including only direct files
shutil.make_archive('model', 'zip', './results', './')


'/content/model.zip'

In [26]:
# Upload the zipped model to Google Drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [29]:

# Copy the zipped model to Google Drive with appropriate naming
def copy_to_drive_with_naming(base_name, drive_path):
    file_index = 0
    while True:
        file_name = f"{base_name}{file_index}.zip"
        destination_path = os.path.join(drive_path, file_name)
        if not os.path.exists(destination_path):
            shutil.copyfile('model.zip', destination_path)
            print(f"Model saved as: {file_name}")
            break
        file_index += 1

# Extract the base model name and dataset short name to use as part of the base name
base_name = f"model_{base_model}_{dataset_short_name}_"
copy_to_drive_path = "/content/drive/My Drive/Models_ML2"
copy_to_drive_with_naming(base_name, copy_to_drive_path)





Model saved as: model_pegasus-large_ECTSum_2.zip


Modell Validierung

In [30]:
!pip install gensim scikit-learn


In [33]:
import json
import pandas as pd
from datasets import Dataset, load_metric
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import random
import os
import shutil
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from sklearn.metrics.pairwise import cosine_similarity

# Load the JSON files
train_json_file_path = '/content/drive/My Drive/Data_ML2/ECTSum_train.jsonl'
val_json_file_path = '/content/drive/My Drive/Data_ML2/ECTSum_val.jsonl'
test_json_file_path = '/content/drive/My Drive/Data_ML2/ECTSum_test.jsonl'

# Read the JSONL files
train_data = []
with open(train_json_file_path, 'r') as file:
    for line in file:
        train_data.append(json.loads(line))

val_data = []
with open(val_json_file_path, 'r') as file:
    for line in file:
        val_data.append(json.loads(line))

test_data = []
with open(test_json_file_path, 'r') as file:
    for line in file:
        test_data.append(json.loads(line))

# Convert to DataFrames for better handling
train_df = pd.DataFrame(train_data)
val_df = pd.DataFrame(val_data)
test_df = pd.DataFrame(test_data)

# Sample 5% of the data
train_df = train_df.sample(frac=0.05, random_state=42)
val_df = val_df.sample(frac=0.05, random_state=42)
test_df = test_df.sample(frac=0.05, random_state=42)

# Convert DataFrames to Datasets
train_dataset = Dataset.from_pandas(train_df)
validation_dataset = Dataset.from_pandas(val_df)
test_dataset = Dataset.from_pandas(test_df)

# Combine the datasets for preprocessing and fine-tuning
combined_df = pd.concat([train_df, val_df, test_df], ignore_index=True)

# Retrieve a random sample
random_index = random.randint(0, len(combined_df) - 1)
initial_sample = combined_df.iloc[random_index]
print("Random sample from the initial dataset (before processing):")
print(initial_sample)

# Convert DataFrame to Dataset
dataset = Dataset.from_pandas(combined_df)

# Load the tokenizer and model
model_name = "google/pegasus-large"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# Define a preprocessing function
def preprocess_function(examples):
    inputs = [doc for doc in examples["input"]]
    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding="max_length")
    labels = tokenizer(text_target=examples["summary"], max_length=128, truncation=True, padding="max_length")
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Apply the preprocessing function to the dataset
processed_dataset = dataset.map(preprocess_function, batched=True)

# Retrieve the processed sample
processed_sample = processed_dataset[random_index]
print("\nRandom sample from the processed dataset (after processing):")
print(processed_sample)

# Generate summaries for the validation set
def generate_summary(text):
    inputs = tokenizer(text, return_tensors="pt", max_length=512, truncation=True)
    summary_ids = model.generate(inputs["input_ids"], max_length=128, min_length=30, num_beams=5, length_penalty=2.0, early_stopping=True)
    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)

# Evaluate the model
rouge = load_metric("rouge")

def evaluate_model(dataset):
    summaries = []
    references = []
    for example in dataset:
        summaries.append(generate_summary(example['input']))
        references.append(example['summary'])
    results = rouge.compute(predictions=summaries, references=references)
    return results

# Calculate ROUGE scores for the validation set
validation_results = evaluate_model(validation_dataset)
print(validation_results)

# Prepare data for Doc2Vec
documents = [TaggedDocument(doc.split(), [i]) for i, doc in enumerate(combined_df['input'])]

# Train Doc2Vec model
doc2vec_model = Doc2Vec(documents, vector_size=50, window=2, min_count=1, workers=4)

# Generate embeddings
def get_embedding(text):
    return doc2vec_model.infer_vector(text.split())

# Calculate cosine similarity
def calculate_cosine_similarity(original, summary):
    original_embedding = get_embedding(original)
    summary_embedding = get_embedding(summary)
    return cosine_similarity([original_embedding], [summary_embedding])[0][0]

# Example usage
example_text = combined_df.iloc[0]['input']
example_summary = generate_summary(example_text)
cosine_sim = calculate_cosine_similarity(example_text, example_summary)
print(f"Cosine Similarity: {cosine_sim}")

# Copy the zipped model to Google Drive with appropriate naming
def copy_to_drive_with_naming(base_name, drive_path):
    file_index = 0
    while True:
        file_name = f"{base_name}{file_index}.zip"
        destination_path = os.path.join(drive_path, file_name)
        if not os.path.exists(destination_path):
            shutil.copyfile('model.zip', destination_path)
            print(f"Model saved as: {file_name}")
            break
        file_index += 1

# Extract the base model name and dataset short name to use as part of the base name
base_model = model_name.split('/')[-1]  # Extract 'pegasus-large' from 'google/pegasus-large'
dataset_short_name = "ECTSum"  # Shortened name for your dataset
base_name = f"model_{base_model}_{dataset_short_name}_"
copy_to_drive_path = "/content/drive/My Drive/Models_ML2"
copy_to_drive_with_naming(base_name, copy_to_drive_path)


Random sample from the initial dataset (before processing):
ticker                                                   RYN
quarter                                                   q2
year                                                    2021
input      They are also referenced on Page 2 of our fina...
summary    q2 pro forma earnings per share $0.22.\nq2 ear...
Name: 94, dtype: object


Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-large and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/121 [00:00<?, ? examples/s]


Random sample from the processed dataset (after processing):
{'ticker': 'RYN', 'quarter': 'q2', 'year': '2021', 'input': "They are also referenced on Page 2 of our financial supplement.\nWith that, let's start our teleconference with opening comments from Dave Nunes, President and CEO.\nFirst, I'll make some high-level comments before turning it back over to Mark McHugh, Senior Vice President and Chief Financial Officer to review our consolidated financial results.\nThen I'll ask Doug Long, our Senior Vice President of Forest Resources, to comment on our U.S. and New Zealand timber results.\nAnd following the review of our timber segments, Mark will discuss our real estate results as well as our outlook for the balance of 2021.\nWe are pleased to report that the encouraging momentum we experienced across all our businesses to start 2021 continued into the second quarter.\nSpecifically, we generated adjusted EBITDA of $95 million and pro forma earnings per share of $0.22 per share.\nAd

/usr/local/lib/python3.10/dist-packages/datasets/load.py:759: FutureWarning: The repository for rouge contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.2/metrics/rouge/rouge.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


{'rouge1': AggregateScore(low=Score(precision=0.10069542522141246, recall=0.20626689576105772, fmeasure=0.13011459345259102), mid=Score(precision=0.15735082643832396, recall=0.27128224792524913, fmeasure=0.18269335043161355), high=Score(precision=0.22288121689797957, recall=0.34846039609415697, fmeasure=0.24301559253617208)), 'rouge2': AggregateScore(low=Score(precision=0.01700599515501766, recall=0.03327044284619749, fmeasure=0.021160726890271826), mid=Score(precision=0.03642339764013851, recall=0.0655121620582147, fmeasure=0.04248577369142957), high=Score(precision=0.058641066954941166, recall=0.10484322373568697, fmeasure=0.06793965972585275)), 'rougeL': AggregateScore(low=Score(precision=0.06613850627618058, recall=0.13387285242268207, fmeasure=0.08497499123344793), mid=Score(precision=0.09405162949308171, recall=0.17367011362615983, fmeasure=0.11168953272097094), high=Score(precision=0.1291017198637173, recall=0.21934301267355805, fmeasure=0.14641965722184475)), 'rougeLsum': Aggre

Validation of full set

In [32]:
import json
import pandas as pd
from datasets import Dataset, load_metric
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import random
import os
import shutil
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from sklearn.metrics.pairwise import cosine_similarity

# Load the JSON files
train_json_file_path = '/content/drive/My Drive/Data_ML2/ECTSum_train.jsonl'
val_json_file_path = '/content/drive/My Drive/Data_ML2/ECTSum_val.jsonl'
test_json_file_path = '/content/drive/My Drive/Data_ML2/ECTSum_test.jsonl'

# Read the JSONL files
train_data = []
with open(train_json_file_path, 'r') as file:
    for line in file:
        train_data.append(json.loads(line))

val_data = []
with open(val_json_file_path, 'r') as file:
    for line in file:
        val_data.append(json.loads(line))

test_data = []
with open(test_json_file_path, 'r') as file:
    for line in file:
        test_data.append(json.loads(line))

# Convert to DataFrames for better handling
train_df = pd.DataFrame(train_data)
val_df = pd.DataFrame(val_data)
test_df = pd.DataFrame(test_data)

# Convert DataFrames to Datasets
train_dataset = Dataset.from_pandas(train_df)
validation_dataset = Dataset.from_pandas(val_df)
test_dataset = Dataset.from_pandas(test_df)

# Combine the datasets for preprocessing and fine-tuning
combined_df = pd.concat([train_df, val_df, test_df], ignore_index=True)

# Retrieve a random sample
random_index = random.randint(0, len(combined_df) - 1)
initial_sample = combined_df.iloc[random_index]
print("Random sample from the initial dataset (before processing):")
print(initial_sample)

# Convert DataFrame to Dataset
dataset = Dataset.from_pandas(combined_df)

# Load the tokenizer and model
model_name = "google/pegasus-large"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# Define a preprocessing function
def preprocess_function(examples):
    inputs = [doc for doc in examples["input"]]
    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding="max_length")
    labels = tokenizer(text_target=examples["summary"], max_length=128, truncation=True, padding="max_length")
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Apply the preprocessing function to the dataset
processed_dataset = dataset.map(preprocess_function, batched=True)

# Retrieve the processed sample
processed_sample = processed_dataset[random_index]
print("\nRandom sample from the processed dataset (after processing):")
print(processed_sample)

# Generate summaries for the validation set
def generate_summary(text):
    inputs = tokenizer(text, return_tensors="pt", max_length=512, truncation=True)
    summary_ids = model.generate(inputs["input_ids"], max_length=128, min_length=30, num_beams=5, length_penalty=2.0, early_stopping=True)
    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)

# Evaluate the model
rouge = load_metric("rouge")

def evaluate_model(dataset):
    summaries = []
    references = []
    for example in dataset:
        summaries.append(generate_summary(example['input']))
        references.append(example['summary'])
    results = rouge.compute(predictions=summaries, references=references)
    return results

# Calculate ROUGE scores for the validation set
validation_results = evaluate_model(validation_dataset)
print(validation_results)

# Prepare data for Doc2Vec
documents = [TaggedDocument(doc.split(), [i]) for i, doc in enumerate(combined_df['input'])]

# Train Doc2Vec model
doc2vec_model = Doc2Vec(documents, vector_size=50, window=2, min_count=1, workers=4)

# Generate embeddings
def get_embedding(text):
    return doc2vec_model.infer_vector(text.split())

# Calculate cosine similarity
def calculate_cosine_similarity(original, summary):
    original_embedding = get_embedding(original)
    summary_embedding = get_embedding(summary)
    return cosine_similarity([original_embedding], [summary_embedding])[0][0]

# Example usage
example_text = combined_df.iloc[0]['input']
example_summary = generate_summary(example_text)
cosine_sim = calculate_cosine_similarity(example_text, example_summary)
print(f"Cosine Similarity: {cosine_sim}")

# Copy the zipped model to Google Drive with appropriate naming
def copy_to_drive_with_naming(base_name, drive_path):
    file_index = 0
    while True:
        file_name = f"{base_name}{file_index}.zip"
        destination_path = os.path.join(drive_path, file_name)
        if not os.path.exists(destination_path):
            shutil.copyfile('model.zip', destination_path)
            print(f"Model saved as: {file_name}")
            break
        file_index += 1

# Extract the base model name and dataset short name to use as part of the base name
base_model = model_name.split('/')[-1]  # Extract 'pegasus-large' from 'google/pegasus-large'
dataset_short_name = "ECTSum"  # Shortened name for your dataset
base_name = f"model_{base_model}_{dataset_short_name}_"
copy_to_drive_path = "/content/drive/My Drive/Models_ML2"
copy_to_drive_with_naming(base_name, copy_to_drive_path)


Random sample from the initial dataset (before processing):
ticker                                                   ARI
quarter                                                   q4
year                                                    2020
input      We hope that everyone listening continues to b...
summary    compname reports q4 earnings per share $0.23.\...
Name: 102, dtype: object


Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-large and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/2425 [00:00<?, ? examples/s]


Random sample from the processed dataset (after processing):
{'ticker': 'ARI', 'quarter': 'q4', 'year': '2020', 'input': 'We hope that everyone listening continues to be safe and healthy as we work through the challenges related to the pandemic.\nIt is impossible to review ARI\'s 2020 performance or discuss current market conditions and the implications for ARI\'s future strategic priorities without acknowledging the initial and ongoing impact of the pandemic.\nUltimately, the real estate market resides at the intersection of the economy and the capital markets.\nTo frame my comments in the appropriate context, it is important to note that despite initial concerns expressed by those who viewed the pandemic through the lens of the Global Financial Crisis, over the past 11 months the capital markets have remained functioning and experienced an historically rapid recovery.\nHowever, overall economic performance is recovering slowly, and the ultimate trajectory of the economy will depend 

<ipython-input-32-14ff11773fae>:82: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  rouge = load_metric("rouge")
/usr/local/lib/python3.10/dist-packages/datasets/load.py:759: FutureWarning: The repository for rouge contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.2/metrics/rouge/rouge.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


KeyboardInterrupt: 